# Estadísticas del etiquetado de emociones

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import json

In [2]:
with open('./jazmin_flores_11_08_21.json') as json_file:
    jazmin = json.load(json_file)
    
with open('./mara_vazquez_11_08_21.json') as json_file:
    mara = json.load(json_file)
    
with open('./omar_hernandez_11_08_21.json') as json_file:
    omar = json.load(json_file)

In [3]:
psicologos = {'jazmin':jazmin, 'mara':mara, 'omar':omar}

## 1) Del total de los audios de una semana, en cuántos hubo updates?
### Estos datos corresponden a una misma semana?

In [88]:
# cuántos audios hizo cada uno esta semana
for k in psicologos.keys():
    data = psicologos[k]
    

In [89]:
# en cuántos hubo updates

import datetime
format = "%Y-%m-%d %H:%M:%S"

n_labels = {} # cuántos etiquetados hizo la persona
update_ratio = {} # ratio cantidad de updates sobre cantidad de etiquetas

for k in psicologos.keys():
    data = psicologos[k]
    
    n_updates = 0
    for i in range(len(data)):
        created_str = data[i]['annotations'][0]['created_at'].replace('T', ' ')[:-8]
        updated_str = data[i]['annotations'][0]['updated_at'].replace('T', ' ')[:-8]
        created_time = datetime.datetime.strptime(created_str, format)
        updated_time = datetime.datetime.strptime(updated_str, format)
        time_interval = (updated_time - created_time).total_seconds()
        
        if time_interval>0:
            n_updates = n_updates+1
            
    n_labels[k] = len(data)
    update_ratio[k] = n_updates/float(len(data))
    print(k, 'etiquetó', len(data), 'audios, con', n_updates, 'updates')

jazmin etiquetó 62 audios, con 0 updates
mara etiquetó 815 audios, con 0 updates
omar etiquetó 901 audios, con 5 updates


In [90]:
# grafico el porcentaje de audios revisados

fig = px.histogram(x=update_ratio.keys(), y=update_ratio.values(), title="Ratio de updates sobre cantidad de audios", width=600, height=400).update_xaxes(categoryorder="total descending")
fig.update_layout(xaxis_title="Psicólogo", yaxis_title="Ratio")
fig.show()

## 2) Cuántas etiquetas por semana hubo?

In [91]:
fig = px.histogram(x=n_labels.keys(), y=n_labels.values(), title="Cantidad de etiquetas por persona en la semana", width=600, height=400).update_xaxes(categoryorder="total descending")
fig.update_layout(xaxis_title="Psicólogo", yaxis_title="Cantidad de etiquetas")
fig.show()

## 3) Distribución de confianza por persona

In [92]:
for k in psicologos.keys():
    data = psicologos[k]
    confidence = []
    for i in range(len(data)):
        c = data[i]['annotations'][0]['result'][2]['value']['choices'][0]
        confidence.append(c)
    fig = px.histogram(confidence, x=confidence, title=k, width=600, height=400)
    fig.show()

## 4) Tiempo total de etiquetado por persona
Sumar lead_time de todos los audios, por persona, por semana

In [93]:
lead_time = {} # tiempo total por persona
for k in psicologos.keys():
    data = psicologos[k]
    l = [] # lead_time de cada audio
    for i in range(len(data)):
        l.append(data[i]['annotations'][0]['lead_time'])
    lead_time[k] = sum(l)/3600.0
    
fig = px.histogram(x=lead_time.keys(), y=lead_time.values(), title='Tiempo total de etiquetado por persona (en horas)', width=600, height=400).update_xaxes(categoryorder="total descending")
fig.update_layout(xaxis_title="Psicólogo", yaxis_title="Tiempo (h)")
fig.show()

## 5) Cantidad de updates por cada emoción
Creo que lo más práctico es armar un df por cada persona

In [19]:
import datetime
format = "%Y-%m-%d %H:%M:%S"

for k in psicologos.keys():
    data = psicologos[k]
    print(k)
    
    emotion = []
    df = pd.DataFrame(columns = ['emotion', 'updated'])
    for i in range(len(data)):
        emotion = data[i]['annotations'][0]['result'][0]['value']['choices'][0]
        
        created_str = data[i]['annotations'][0]['created_at'].replace('T', ' ')[:-8]
        updated_str = data[i]['annotations'][0]['updated_at'].replace('T', ' ')[:-8]
        created_time = datetime.datetime.strptime(created_str, format)
        updated_time = datetime.datetime.strptime(updated_str, format)
        time_interval = (updated_time - created_time).total_seconds()
        
        updated = 0
        if time_interval>0:
            updated = 1
            
        # agrego una fila
        df.loc[len(df)] = [emotion, updated]
    
    print(df.groupby(['emotion']).count())

jazmin
          updated
emotion          
Alegría         4
Alta            2
Baja            6
Ira             2
Media          14
Miedo           3
Neutro         30
Tristeza        1
mara
              updated
emotion              
Alegría           110
Baja                1
Disgusto          259
Ira               110
Miedo             112
Neutro            111
No_valorable       88
Sorpresa           13
Tristeza           11
omar
              updated
emotion              
Alegría           134
Alta                1
Disgusto          235
Ira               121
Miedo             119
Neutro            134
No_valorable      105
Sorpresa           34
Tristeza           18


### En el campo "emotion" aparece mezclada la intensidad

In [17]:
data=jazmin
i=9
emotion = data[i]['annotations'][0]['result']#[0]['value']['choices'][0]

emotion

[{'value': {'choices': ['Media']},
  'id': 'guemU2hYxn',
  'from_name': 'intensity',
  'to_name': 'audio',
  'type': 'choices'},
 {'value': {'choices': ['Alta']},
  'id': 'Y7mzxU9OFo',
  'from_name': 'confidence',
  'to_name': 'audio',
  'type': 'choices'},
 {'value': {'choices': ['Neutro']},
  'id': 'rtcsYfTzTm',
  'from_name': 'emotion',
  'to_name': 'audio',
  'type': 'choices'}]

In [18]:
data=jazmin
i=0
emotion = data[i]['annotations'][0]['result']#[0]['value']['choices'][0]

emotion

[{'value': {'choices': ['Neutro']},
  'id': 'qe9nPOTNIo',
  'from_name': 'emotion',
  'to_name': 'audio',
  'type': 'choices'},
 {'value': {'choices': ['Media']},
  'id': '2Uo7fN05SN',
  'from_name': 'intensity',
  'to_name': 'audio',
  'type': 'choices'},
 {'value': {'choices': ['Alta']},
  'id': 'MCgbIxbTI8',
  'from_name': 'confidence',
  'to_name': 'audio',
  'type': 'choices'}]

# Problema:
### El orden de los campos no es consistente
O lo estandarizamos, o tengo que basar la búsqueda en el nombre del campo